<a href="https://colab.research.google.com/github/apwvt/S2DC-RL-Project/blob/main/S2DC%20Reinforcement%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pull Project Code**

---
We first get the latest version of the project code.


In [1]:
from os import path
if path.exists("S2DC-RL-Project"):
  ! cd S2DC-RL-Project && git pull
else:
  ! git clone https://github.com/apwvt/S2DC-RL-Project.git

Cloning into 'S2DC-RL-Project'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 92 (delta 35), reused 67 (delta 19), pack-reused 0
Unpacking objects: 100% (92/92), done.


In [22]:
# We add the repository folder to the module path.
import sys
sys.path.insert(1, "/content/S2DC-RL-Project")

**Mount Drive**

---
We mount a shared drive to have a place to save checkpoints.

In [2]:
from google.colab import drive
drive.mount('/content/Drive')

%set_env CPFOLDER=/content/Drive/Shareddrives/CS_ML_ENV/colab_env/S2DC-RL-Project

Mounted at /content/Drive
env: CPFOLDER=/content/Drive/Shareddrives/CS_ML_ENV/colab_env/S2DC-RL-Project


**Setup Dependencies**

---
We install our repository as a local package.

In [ ]:
% cd S2DC-RL-Project
! pip install -e .  # Local module
! pip install ray   # Distributed computing package

**Run Project Code**

---

Any project files can be loaded from here and used in programs.

In [ ]:
from muzero_collab.muzero import MuZero

muzero = MuZero("battle")